___

### les bases
___
* sélectionner un estimateur et préciser ses hyperparamètres : modele = LinearRegression(.........)
* Entrainer le modèle sur les données X, y (divisées en 2 tableaux Numpy) : model.fit(X, y)
* Evaluer le modèle : model.score(X, y)
* Utiliser le modèle : model.predict(X)

In [54]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [85]:
# df_original = pd.read_csv("données d'origine.csv")
df_original = pd.read_csv("data_cleaned.csv")
df = df_original[['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']]

df['group'] = np.where(df['charges'] < 4746.344000, '1', 
                             np.where(df['charges'] < 9386.161300, '2', 
                             np.where(df['charges'] < 16657.717450, '3', '4')))

# df = df[df['charges'] < 49000]
# df_group['group'] = np.where(df_group['charges'] < 4719.736550, '1', 
                            #  np.where(df_group['charges'] < 9301.893550, '2', 
                            #  np.where(df_group['charges'] < 16297.846000, '3', '4')))

mapping_sex = {'male': 0, 'female': 1}
mapping_smoker = {'yes': 1, 'no': 0}

# df['sex'] = df['sex'].map(mapping_sex)
# df['smoker'] = df['smoker'].map(mapping_smoker)
df['northeast'] = 0
df['southeast'] = 0
df['northwest'] = 0
df['southwest'] = 0
df.loc[df['region'] == 'northeast', 'northeast'] = 1
df.loc[df['region'] == 'southeast', 'southeast'] = 1
df.loc[df['region'] == 'northwest', 'northwest'] = 1
df.loc[df['region'] == 'southwest', 'southwest'] = 1

df.loc[:, 'smoker_bmi'] = df['smoker'] * df['bmi']

# nettoyage
df = df[['age', 'sex', 'bmi', 'children', 'smoker', 'smoker_bmi', 'northeast', 'southeast', 'northwest', 'southwest', 'charges', 'group']]

display(df)
# display(df.isnull().sum())

/tmp/ipykernel_161742/1558382575.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['group'] = np.where(df['charges'] < 4746.344000, '1',


,age,sex,bmi,children,smoker,smoker_bmi,northeast,southeast,northwest,southwest,charges,group
0,19,1,27.900,0,1,27.90,0,0,0,1,16884.92400,4
1,18,0,33.770,1,0,0.00,0,1,0,0,1725.55230,1
2,28,0,33.000,3,0,0.00,0,1,0,0,4449.46200,1
3,33,0,22.705,0,0,0.00,0,0,1,0,21984.47061,4
4,32,0,28.880,0,0,0.00,0,0,1,0,3866.85520,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1332,50,0,30.970,3,0,0.00,0,0,1,0,10600.54830,3
1333,18,1,31.920,0,0,0.00,1,0,0,0,2205.98080,1
1334,18,1,36.850,0,0,0.00,0,1,0,0,1629.83350,1
1335,21,1,25.800,0,0,0.00,0,0,0,1,2007.94500,1


In [95]:
from sklearn.model_selection import train_test_split # pour diviser le jeu de données initial en 1 jeu d'entrainement (X_train, y_train) et un jeu de test (X_test, y_test)
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

y = df[['charges']]
X = df[['age', 'sex', 'bmi', 'children', 'smoker', 'smoker_bmi', 'northeast', 'southeast', 'northwest', 'southwest']]
X = df[['bmi', 'children', 'smoker', 'smoker_bmi']]
X_dummies = pd.get_dummies(df[['age', 'bmi', 'smoker', 'smoker_bmi', 'children']], columns=['age', 'bmi', 'smoker', 'smoker_bmi', 'children'], drop_first=True)
# display(X)

X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, shuffle=True, train_size=0.85, random_state=42, stratify=X_dummies['smoker_1'])
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_train)
print(model.score(X_train, y_train), r2_score(y_train, predictions))

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_train)
print(model.score(X_train, y_train), r2_score(y_train, predictions))

# model = LinearRegression()
# model.fit(X_train, y_train)
# model.score(X_train, y_train)

# step = np.arange(0,1.1,1)
# param_grid = {'alpha': np.arange(0,1),
#               'l1_ratio': np.arange(0,1)}
# grid = GridSearchCV(ElasticNet(), param_grid, cv = 42)
# grid.fit(X_train, y_train)

# print('')
# print('_______________________________________________________')
# print('###BEST### >>>', round(grid.best_score_, 2))

0.9343148567500592 0.9343148567500592
0.7355691202476615 0.7355691202476615
